In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer,BertForMaskedLM
from datasets import load_dataset
from transformers import TrainingArguments, DataCollatorForLanguageModeling,PreTrainedTokenizerFast
import numpy as np
import evaluate
from transformers import Trainer
from datasets import Dataset
from tokenizers import Tokenizer
import os
from itertools import product
import json
import wandb
import torch 
import time
import gc
wandb.init(mode="disabled")

In [ ]:
file_to_train='strict_model_individual_datasets'
path='./SlovakBabyLM/Curricullum_learning/'
files=next(os.walk(f'{path}saved_model/{file_to_train}'))[1]

## sentiment analysis

In [3]:
dataset = load_dataset("DGurgurov/slovak_sa")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 3560
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 522
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 1042
    })
})

In [4]:
learning_rate = [5e-5,3e-5,1e-5]
number_of_epochs = [5, 7,10]

combined_params = [{'learning_rate': lr, 'epoch': ep} for lr, ep in product(learning_rate, number_of_epochs)]

In [14]:
combined_params

[{'learning_rate': 5e-05, 'epoch': 5},
 {'learning_rate': 5e-05, 'epoch': 7},
 {'learning_rate': 5e-05, 'epoch': 10},
 {'learning_rate': 3e-05, 'epoch': 5},
 {'learning_rate': 3e-05, 'epoch': 7},
 {'learning_rate': 3e-05, 'epoch': 10},
 {'learning_rate': 1e-05, 'epoch': 5},
 {'learning_rate': 1e-05, 'epoch': 7},
 {'learning_rate': 1e-05, 'epoch': 10}]

In [6]:
files

['randomly_selected_subs', 'randomly_selected_wiki_full']

In [5]:
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")
accuracy_metric = evaluate.load("accuracy")

In [ ]:
results_eval = {file_model: [] for file_model in files}
for parameters in combined_params:
    learning_rate=parameters['learning_rate']
    number_of_epochs=parameters['epoch']
    for file_to_load in files:
        if file_to_load:
            save_path = f"{path}saved_model/{file_to_train}/{file_to_load}"
            tokenizer = AutoTokenizer.from_pretrained(save_path)
            model = AutoModelForSequenceClassification.from_pretrained(save_path, local_files_only=True, num_labels=2)

            def preprocess_function(examples):
                return tokenizer(examples["text"], padding="max_length", truncation=True)

            tokenized_datasets = dataset.map(preprocess_function, batched=True)


            training_args = TrainingArguments(
            output_dir=f"{path}saved_model/{file_to_train}/{file_to_load}/testing_sentiment/results",
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=learning_rate,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=number_of_epochs,
            weight_decay=0.01,
            logging_dir=f"{path}saved_model/{file_to_train}/{file_to_load}/testing_sentiment/logs",
        )



            def compute_metrics(eval_pred):
                logits, labels = eval_pred
                predictions = np.argmax(logits, axis=-1)
                precision = precision_metric.compute(predictions=predictions, references=labels, average="weighted")
                recall = recall_metric.compute(predictions=predictions, references=labels, average="weighted")
                f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
                accuracy=accuracy_metric.compute(predictions=predictions, references=labels)
                return {
                "accuracy": accuracy["accuracy"],
                "precision": precision["precision"],
                "recall": recall["recall"],
                "f1": f1["f1"],
                }



            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=tokenized_datasets["train"],
                eval_dataset=tokenized_datasets["validation"],
                tokenizer=tokenizer,
                compute_metrics=compute_metrics
            )

            trainer.train()
            results=trainer.evaluate(tokenized_datasets["test"])
            for key in ['eval_samples_per_second','eval_runtime','eval_steps_per_second','epoch']:
                results.pop(key, None)
            results['parameters']=[learning_rate,number_of_epochs]
            results_eval[file_to_load].append(results)
            torch.cuda.empty_cache()
            gc.collect()
            time.sleep(7)

import json
with open(f'{path}results_evaluation/{file_to_train}/output_sentiment.json', 'w') as f:
    json.dump(results_eval, f)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /data/lubosk/diploma_thesis/saved_model/strict_model_individual_datasets/randomly_selected_subs and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/data/lubosk/.local/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_7

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.421503,0.867816,0.753105,0.867816,0.806401
2,No log,0.394808,0.867816,0.753105,0.867816,0.806401
3,No log,0.384534,0.867816,0.753105,0.867816,0.806401
4,No log,0.313776,0.898467,0.887617,0.898467,0.889571
5,0.416300,0.300198,0.904215,0.894499,0.904215,0.895368


/data/lubosk/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/lubosk/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/data/lubosk/.local/lib/python3.8/site-packages/torch/nn/parallel/data_parallel.py:34: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(d

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /data/lubosk/diploma_thesis/saved_model/strict_model_individual_datasets/randomly_selected_wiki_full and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/data/lubosk/.local/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipyker

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.433709,0.867816,0.753105,0.867816,0.806401
2,No log,0.362477,0.867816,0.753105,0.867816,0.806401
3,No log,0.284292,0.908046,0.910400,0.908046,0.909137
4,No log,0.261830,0.917625,0.917126,0.917625,0.917369
5,0.373800,0.248753,0.925287,0.921873,0.925287,0.923049


...

/data/lubosk/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/lubosk/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/data/lubosk/.local/lib/python3.8/site-packages/torch/nn/parallel/data_parallel.py:34: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(d

/data/lubosk/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /data/lubosk/diploma_thesis/saved_model/strict_model_individual_datasets/randomly_selected_wiki_full and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/data/lubosk/.local/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The 

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.553747,0.867816,0.753105,0.867816,0.806401
2,No log,0.501240,0.867816,0.753105,0.867816,0.806401
3,No log,0.471661,0.867816,0.753105,0.867816,0.806401
4,No log,0.450704,0.867816,0.753105,0.867816,0.806401
5,0.527600,0.436827,0.867816,0.753105,0.867816,0.806401
6,0.527600,0.427446,0.867816,0.753105,0.867816,0.806401
7,0.527600,0.421320,0.867816,0.753105,0.867816,0.806401
8,0.527600,0.417452,0.867816,0.753105,0.867816,0.806401
9,0.448700,0.415168,0.867816,0.753105,0.867816,0.806401
10,0.448700,0.414483,0.867816,0.753105,0.867816,0.806401


/data/lubosk/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/lubosk/.local/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/data/lubosk/.local/lib/python3.8/site-packages/torch/nn/parallel/data_parallel.py:34: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(d

/data/lubosk/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import json
with open(f'{path}output_sentiment', 'w') as f:
    json.dump(results_eval, f)